# Integração entre as Bases RD Station e Exact Sales

> Aqui vamos apontar e resolver os problemas de integração entre as bases [RD Station](https://plugcrm.net/) e [Exact Sales](https://app.exactspotter.com/Account/NormalLogin?email=fernando%40absolutasaude.com.br), duas bases de CRM utilizadas pela empresa [Absoluta Saúde](https://absolutasaude.com.br/)

## O Problema

O problema a ser resolvido está relacionado à importação da base de _leads_ da base RD para a base Exact, e são eles:

- **Importação incompleta do número de _leads_**: não foram importados todos os leads para a base Exact, sendo que temos 18197 _leads_ na base Exact e 32346 _leads_ na base RD. Assim, _temos 14149 leads que não foram importados para a base Exact_;
- **Importação incompleta das informações dos _leads_**: alguns _leads_ não foram importados com todas as informações para a base Exact, mesmo tendo essas informações na base RD. Alguns casos como: ZANON TECNOLOGIA LTDA ME, VIDAM BRASIL AG DE TURISMO E VIAGENS LTDA entre outras.
    - Uma possível causa desse proble é que pelo formato de saída da exportação do RD do arquivo `.json` ter o _array_ de objetos `custom_fields`, o Exact pode não reconhecer isso da melhor forma na hora da importação

## [Exact Sales](https://app.exactspotter.com/Account/NormalLogin?email=fernando%40absolutasaude.com.br)

> Temos **18197 leads** nessa base

> - Login: fernando@absolutasaude.com.br
> - Senha: 0i9gky32

### API

- Manual: [Exact Spotter V2 - API v3](https://exactsal.es/apiv3-spotterv2)
- Chave: https://api.exactspotter.com/v3/Leads
- Token: 35fc1bd4-7d6e-4f40-bffd-123d2f46054a

#### Limitações
- Limite de 500 leads por requisição
- Limite de 30 requisições a cada 20 segundos

In [ ]:
# GET Request para leads da Exact

import requests

headers = {
  'Content-Type': 'application/json',
  'token_exact': '35fc1bd4-7d6e-4f40-bffd-123d2f46054a'
}

response = requests.get('https://api.exactspotter.com/v3/Leads', headers=headers)

print(response.status_code)
#print(response.headers)
print(response.json())

In [371]:
# Criando base de dados para o Exact

import requests
import json

pagesErrorExct = []
totalLeadsExct = 0

### Funções
## Função para a requisição da API
def request_API(skipNum):
    url = 'https://api.exactspotter.com/v3/Leads'
    headers = {
      'Content-Type': 'application/json',
      'token_exact': '35fc1bd4-7d6e-4f40-bffd-123d2f46054a'
    }
    params = {
        '$skip': skipNum,
        '$orderby': 'lead'
    }
    
    result = requests.get(url, headers=headers, params=params)
    
    # Se a requisição for um sucesso, retorna o resultado
    if (result.status_code == 200):
        print('Requisição: Sucesso')        
        return result
    
    # Se a requisição falhar, adiciona o número da página à lista de erro
    elif (result.status_code == 400):
        print('Requisição: Falhou')
        pagesErrorExct.append(page)

## Função para remover itens da lista
def remove_items(list):    
    for i in range(0, len(list)):
        # Contando quantos leads tem nessa lista
        global totalLeadsExct
        totalLeadsExct = totalLeadsExct + 1
        
        # Removendo campos desnecessários de cada lead
        list[i].pop('registerDate')
        list[i].pop('updateDate')
        list[i].pop('mktLink')
        list[i].pop('leadProduct')
        list[i].pop('stage')
        list[i].pop('integrationId')        
                    
### Criando a database
listCompleteExct = []

for i in range(0, 30000, 500): # Usando um número bem alto como 50000 para não correr o risco de não trazer todos os leads
    print('Skip atual:', i)
    ## Atualizando a requisição
    responseExct = request_API(i)
    
    if (responseExct):
        ## Transformando em lista (array)
        responseStrExct = json.dumps(responseExct.json()["value"])
        listExct = json.loads(responseStrExct)

        if (listExct):
            print('Skip tem leads')
            ## Removendo alguns itens da lista
            remove_items(listExct)

            ## Juntando as listas
            listCompleteExct.append(listExct)
            jsonDatabaseExct = json.dumps(listCompleteExct)
        
        else:
            print('Skip não tem leads')

### Escrevenddo no arquivo json
jsonFileExct = open("completeDatabase_Exact1.json", "w")
jsonFileExct.write(jsonDatabaseExct)
jsonFileExct.close()

print('Páginas com erro:', pagesErrorExct)
print('Total de leads:', totalLeadsExct)
print("Finalizado")

Skip atual: 0
Requisição: Sucesso
Skip tem leads
Skip atual: 500
Requisição: Sucesso
Skip tem leads
Skip atual: 1000
Requisição: Sucesso
Skip tem leads
Skip atual: 1500
Requisição: Sucesso
Skip tem leads
Skip atual: 2000
Requisição: Sucesso
Skip tem leads
Skip atual: 2500
Requisição: Sucesso
Skip tem leads
Skip atual: 3000
Requisição: Sucesso
Skip tem leads
Skip atual: 3500
Requisição: Sucesso
Skip tem leads
Skip atual: 4000
Requisição: Sucesso
Skip tem leads
Skip atual: 4500
Requisição: Sucesso
Skip tem leads
Skip atual: 5000
Requisição: Sucesso
Skip tem leads
Skip atual: 5500
Requisição: Sucesso
Skip tem leads
Skip atual: 6000
Requisição: Sucesso
Skip tem leads
Skip atual: 6500
Requisição: Sucesso
Skip tem leads
Skip atual: 7000
Requisição: Sucesso
Skip tem leads
Skip atual: 7500
Requisição: Sucesso
Skip tem leads
Skip atual: 8000
Requisição: Sucesso
Skip tem leads
Skip atual: 8500
Requisição: Sucesso
Skip tem leads
Skip atual: 9000
Requisição: Sucesso
Skip tem leads
Skip atual: 9500

## [RD Station](https://plugcrm.net/)

> Temos **32346 leads** nessa base

> - Login: luciane@absolutasaude.com.br
> - Senha: SgAbs76@380

> - Login: f.aspires@hotmail.com
> - Senha: abs020172

### API

- Manual: [Developers RD Station](https://developers.rdstation.com/reference/instruções-e-requisitos)
- Chave: https://crm.rdstation.com/api/v1/organizations?token=5b67dabde7b4bd01146f3d53
- Token: 5b67dabde7b4bd01146f3d53, Conta f.aspires: 5b67dabde7b4bd01146f3d52

#### Limitações
- Limite de 200 leads por requisição
- Limite de 120 requisições por minuto

In [ ]:
# GET Request para leads da RD

import requests

params = {
  'token': '5b67dabde7b4bd01146f3d53'
}

response = requests.get('https://crm.rdstation.com/api/v1/organizations', params=params)

print(response.status_code)
#print(response.headers)
print(response.json())

In [373]:
# Criando base de dados para o RD

import requests
import json

pagesErrorRd = []
totalLeadsRd = 0

### Funções
## Função para a requisição da API
def request_API(id, page):
    url = 'https://crm.rdstation.com/api/v1/organizations'
    params = {
        'token': '5b67dabde7b4bd01146f3d53',
        'page': page,
        'limit': 500,
        'order': 'name',
        'direction': 'asc',
        'user_id': id
    }
    
    result = requests.get(url, params=params)
    
    # Se a requisição for um sucesso, retorna o resultado
    if (result.status_code == 200):
        print('Requisição: Sucesso')        
        return result
    
    # Se a requisição falhar, adiciona o número da página à lista de erro
    elif (result.status_code == 400):
        print('Requisição: Falhou')
        pagesErrorRd.append(page)

## Função para remover itens da lista
def remove_items(list):
    for i in range(0, len(list)):
        # Contando quantos leads tem nessa lista
        global totalLeadsRd
        totalLeadsRd = totalLeadsRd + 1
        
        # Removendo campos desnecessários de cada lead
        list[i].pop('resume')
        list[i].pop('won_count')
        list[i].pop('lost_count')
        list[i].pop('opened_count')
        list[i].pop('paused_count')
        list[i].pop('updated_at')
        list[i].pop('organization_segments')
        
        if len(list[i]["contacts"]):
            list[i]["contacts"][0].pop('notes')
            list[i]["contacts"][0].pop('facebook')
            list[i]["contacts"][0].pop('linkedin')
            list[i]["contacts"][0].pop('skype')
            list[i]["contacts"][0].pop('birthday')
        
        if len(list[i]["custom_fields"]):
            # Limpando o campo CNPJ, deixando somente os números
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CNPJ"):
                    disallowed_characters = "./-"
                    for character in disallowed_characters:
                        if (list[i]["custom_fields"][j]["value"]):
                            list[i]["custom_fields"][j]["value"] = list[i]["custom_fields"][j]["value"].replace(character, "")
            # Excluindo campos desnecessários dos custom fields
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "FIXO 2"):                    
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "FIXO 3"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CELULAR 2"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CELULAR 3"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "EMAIL 2"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "EMAIL 3"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "COMERCIAL 1"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "COMERCIAL 2"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "COMERCIAL 3"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 2  SOCIO 1"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL  3  SOCIO 1"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "NOME SOCIO 2"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CATEGORIA SOCIO 2"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 1  SOCIO 2"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 2  SOCIO 2"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 3  SOCIO 2"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "NOME SOCIO 3"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CATEGORIA SOCIO 3"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 1  SOCIO 3"):
                    del[list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 2  SOCIO 3"):
                    del[list[i]["custom_fields"][j]]

### Usuários
# Tem a requisição para isso, mas não traz todos os usuários
usersId = [
    '5aabc9e0276cfa000172b5e5',
    '5ab552396d3909000c2fef1f',
    '5abccec74b081b000e0cb4fd',
    '5ac3970c5a866b000d0ea164',
    '5ac397a23e9bd3000ccd6aef',
    '5ac3980f3e9bd3000ccd6b78',
    '5ac4ceb9975369000ba86002',
    '5ac512b7ff595e001899fbaf',
    '5acced2c3eb788000b6de43c',
    '5ae210d9b96ff8000bbd3a32',
    '5b0db0582ee7c5000b820966',
    '5b23cd1217762e000b8f1ae8',
    '5b23cdb817762e00148f15ba',
    '5b2b9fbeb73caa0018c70f9a',
    '5b3f6f62fdc6ad000b38336d',
    '5c06879c17a5240010a6a562',
    '5c35f0adabcdfc0021b21ff5',
    '5c38757b7fbef70010b93bd4',
    '5c49a36c2f105c002c560354',
    '5c798c6662345d00105988c3',
    '5e15fed90b64410025ae2327',
    '5e31f0a16094d9001071951c',
    '5ecd7e38692a01002956739f',
    '5ecfc1279580a10024a1816f',
    '5ede4820bec78d0024d99daa',
    '5f0cbf70649f1500296d29b4',
    '5f0f0623356df5002b0cfaaa',
    '5f13056866f067000ebdf6da',
    '5f25f890a5281e0019080727',
    '5f6497bfbb41e200207a9011',
    '5fae972f03d7d80023cf9fa4',
    '5fc628bcca2160001e2527fb',
    '5fc637ad204518000afdeafd',
    '5fc6396aec62040013276d5a',
    '5fc63a9f786e270012c78725',
    '5fc658ebffb234000959302c',
    '5fc65dc4eee8cf0015b2dd89',
    '60a2f17e4899a100161e72a9',
    '6123c1a49f809a000b8d4591',
    '6171921d9edde5001fd0bb6f',
]

### Criando a database
listCompleteRd = []

for i in range(0, len(usersId)):
    print('ID:', i + 1)
    # AQUI TEMOS O ERRO DE LIMITAÇÃO DA API, ACIMA DE 50 PÁGINAS COM 500 POR REQUISIÇÃO RECEBEMOS ERRO
    for j in range(1, 50): # Usando um número bem alto como 50 para não correr o risco de não trazer todos os leads
        print('Página:', j)
        ## Atualizando a requisição
        responseRd = request_API(usersId[i], j)

        if (responseRd):
            ## Transformando em lista (array)
            responseStrRd = json.dumps(responseRd.json()["organizations"])
            listRd = json.loads(responseStrRd)
            
            if (listRd):
                print('Página tem leads')

                ## Removendo alguns itens da lista
                remove_items(listRd)

                ## Juntando as listas
                listCompleteRd.append(listRd)
                jsonDatabaseRd = json.dumps(listCompleteRd)
                
            else:
                print('Página não tem leads')

### Escrevenddo no arquivo json
jsonFileRd = open("completeDatabase_RD1.json", "w")
jsonFileRd.write(jsonDatabaseRd)
jsonFileRd.close()

print('Páginas com erro:', pagesErrorRd)
print('Total de leads:', totalLeadsRd)
print("Finalizado")

ID: 0
Página: 1
Requisição: Sucesso
Página tem leads
Página: 2
Requisição: Sucesso
Página tem leads
Página: 3
Requisição: Sucesso
Página tem leads
Página: 4
Requisição: Sucesso
Página tem leads
Página: 5
Requisição: Sucesso
Página tem leads
Página: 6
Requisição: Sucesso
Página tem leads
Página: 7
Requisição: Sucesso
Página tem leads
Página: 8
Requisição: Sucesso
Página tem leads
Página: 9
Requisição: Sucesso
Página tem leads
Página: 10
Requisição: Sucesso
Página tem leads
Página: 11
Requisição: Sucesso
Página tem leads
Página: 12
Requisição: Sucesso
Página tem leads
Página: 13
Requisição: Sucesso
Página tem leads
Página: 14
Requisição: Sucesso
Página tem leads
Página: 15
Requisição: Sucesso
Página tem leads
Página: 16
Requisição: Sucesso
Página tem leads
Página: 17
Requisição: Sucesso
Página tem leads
Página: 18
Requisição: Sucesso
Página tem leads
Página: 19
Requisição: Sucesso
Página tem leads
Página: 20
Requisição: Sucesso
Página tem leads
Página: 21
Requisição: Sucesso
Página tem l

Requisição: Sucesso
Página não tem leads
Página: 21
Requisição: Sucesso
Página não tem leads
Página: 22
Requisição: Sucesso
Página não tem leads
Página: 23
Requisição: Sucesso
Página não tem leads
Página: 24
Requisição: Sucesso
Página não tem leads
Página: 25
Requisição: Sucesso
Página não tem leads
Página: 26
Requisição: Sucesso
Página não tem leads
Página: 27
Requisição: Sucesso
Página não tem leads
Página: 28
Requisição: Sucesso
Página não tem leads
Página: 29
Requisição: Sucesso
Página não tem leads
Página: 30
Requisição: Sucesso
Página não tem leads
Página: 31
Requisição: Sucesso
Página não tem leads
Página: 32
Requisição: Sucesso
Página não tem leads
Página: 33
Requisição: Sucesso
Página não tem leads
Página: 34
Requisição: Sucesso
Página não tem leads
Página: 35
Requisição: Sucesso
Página não tem leads
Página: 36
Requisição: Sucesso
Página não tem leads
Página: 37
Requisição: Sucesso
Página não tem leads
Página: 38
Requisição: Sucesso
Página não tem leads
Página: 39
Requisição: 

Requisição: Sucesso
Página não tem leads
Página: 33
Requisição: Sucesso
Página não tem leads
Página: 34
Requisição: Sucesso
Página não tem leads
Página: 35
Requisição: Sucesso
Página não tem leads
Página: 36
Requisição: Sucesso
Página não tem leads
Página: 37
Requisição: Sucesso
Página não tem leads
Página: 38
Requisição: Sucesso
Página não tem leads
Página: 39
Requisição: Sucesso
Página não tem leads
Página: 40
Requisição: Sucesso
Página não tem leads
Página: 41
Requisição: Sucesso
Página não tem leads
Página: 42
Requisição: Sucesso
Página não tem leads
Página: 43
Requisição: Sucesso
Página não tem leads
Página: 44
Requisição: Sucesso
Página não tem leads
Página: 45
Requisição: Sucesso
Página não tem leads
Página: 46
Requisição: Sucesso
Página não tem leads
Página: 47
Requisição: Sucesso
Página não tem leads
Página: 48
Requisição: Sucesso
Página não tem leads
Página: 49
Requisição: Sucesso
Página não tem leads
ID: 7
Página: 1
Requisição: Sucesso
Página tem leads
Página: 2
Requisição: 

Requisição: Sucesso
Página não tem leads
Página: 45
Requisição: Sucesso
Página não tem leads
Página: 46
Requisição: Sucesso
Página não tem leads
Página: 47
Requisição: Sucesso
Página não tem leads
Página: 48
Requisição: Sucesso
Página não tem leads
Página: 49
Requisição: Sucesso
Página não tem leads
ID: 10
Página: 1
Requisição: Sucesso
Página tem leads
Página: 2
Requisição: Sucesso
Página não tem leads
Página: 3
Requisição: Sucesso
Página não tem leads
Página: 4
Requisição: Sucesso
Página não tem leads
Página: 5
Requisição: Sucesso
Página não tem leads
Página: 6
Requisição: Sucesso
Página não tem leads
Página: 7
Requisição: Sucesso
Página não tem leads
Página: 8
Requisição: Sucesso
Página não tem leads
Página: 9
Requisição: Sucesso
Página não tem leads
Página: 10
Requisição: Sucesso
Página não tem leads
Página: 11
Requisição: Sucesso
Página não tem leads
Página: 12
Requisição: Sucesso
Página não tem leads
Página: 13
Requisição: Sucesso
Página não tem leads
Página: 14
Requisição: Sucess

Requisição: Sucesso
Página não tem leads
Página: 7
Requisição: Sucesso
Página não tem leads
Página: 8
Requisição: Sucesso
Página não tem leads
Página: 9
Requisição: Sucesso
Página não tem leads
Página: 10
Requisição: Sucesso
Página não tem leads
Página: 11
Requisição: Sucesso
Página não tem leads
Página: 12
Requisição: Sucesso
Página não tem leads
Página: 13
Requisição: Sucesso
Página não tem leads
Página: 14
Requisição: Sucesso
Página não tem leads
Página: 15
Requisição: Sucesso
Página não tem leads
Página: 16
Requisição: Sucesso
Página não tem leads
Página: 17
Requisição: Sucesso
Página não tem leads
Página: 18
Requisição: Sucesso
Página não tem leads
Página: 19
Requisição: Sucesso
Página não tem leads
Página: 20
Requisição: Sucesso
Página não tem leads
Página: 21
Requisição: Sucesso
Página não tem leads
Página: 22
Requisição: Sucesso
Página não tem leads
Página: 23
Requisição: Sucesso
Página não tem leads
Página: 24
Requisição: Sucesso
Página não tem leads
Página: 25
Requisição: Suc

...

Requisição: Sucesso
Página não tem leads
Página: 3
Requisição: Sucesso
Página não tem leads
Página: 4
Requisição: Sucesso
Página não tem leads
Página: 5
Requisição: Sucesso
Página não tem leads
Página: 6
Requisição: Sucesso
Página não tem leads
Página: 7
Requisição: Sucesso
Página não tem leads
Página: 8
Requisição: Sucesso
Página não tem leads
Página: 9
Requisição: Sucesso
Página não tem leads
Página: 10
Requisição: Sucesso
Página não tem leads
Página: 11
Requisição: Sucesso
Página não tem leads
Página: 12
Requisição: Sucesso
Página não tem leads
Página: 13
Requisição: Sucesso
Página não tem leads
Página: 14
Requisição: Sucesso
Página não tem leads
Página: 15
Requisição: Sucesso
Página não tem leads
Página: 16
Requisição: Sucesso
Página não tem leads
Página: 17
Requisição: Sucesso
Página não tem leads
Página: 18
Requisição: Sucesso
Página não tem leads
Página: 19
Requisição: Sucesso
Página não tem leads
Página: 20
Requisição: Sucesso
Página não tem leads
Página: 21
Requisição: Sucesso

Requisição: Sucesso
Página não tem leads
Página: 15
Requisição: Sucesso
Página não tem leads
Página: 16
Requisição: Sucesso
Página não tem leads
Página: 17
Requisição: Sucesso
Página não tem leads
Página: 18
Requisição: Sucesso
Página não tem leads
Página: 19
Requisição: Sucesso
Página não tem leads
Página: 20
Requisição: Sucesso
Página não tem leads
Página: 21
Requisição: Sucesso
Página não tem leads
Página: 22
Requisição: Sucesso
Página não tem leads
Página: 23
Requisição: Sucesso
Página não tem leads
Página: 24
Requisição: Sucesso
Página não tem leads
Página: 25
Requisição: Sucesso
Página não tem leads
Página: 26
Requisição: Sucesso
Página não tem leads
Página: 27
Requisição: Sucesso
Página não tem leads
Página: 28
Requisição: Sucesso
Página não tem leads
Página: 29
Requisição: Sucesso
Página não tem leads
Página: 30
Requisição: Sucesso
Página não tem leads
Página: 31
Requisição: Sucesso
Página não tem leads
Página: 32
Requisição: Sucesso
Página não tem leads
Página: 33
Requisição: 

Requisição: Sucesso
Página não tem leads
Página: 26
Requisição: Sucesso
Página não tem leads
Página: 27
Requisição: Sucesso
Página não tem leads
Página: 28
Requisição: Sucesso
Página não tem leads
Página: 29
Requisição: Sucesso
Página não tem leads
Página: 30
Requisição: Sucesso
Página não tem leads
Página: 31
Requisição: Sucesso
Página não tem leads
Página: 32
Requisição: Sucesso
Página não tem leads
Página: 33
Requisição: Sucesso
Página não tem leads
Página: 34
Requisição: Sucesso
Página não tem leads
Página: 35
Requisição: Sucesso
Página não tem leads
Página: 36
Requisição: Sucesso
Página não tem leads
Página: 37
Requisição: Sucesso
Página não tem leads
Página: 38
Requisição: Sucesso
Página não tem leads
Página: 39
Requisição: Sucesso
Página não tem leads
Página: 40
Requisição: Sucesso
Página não tem leads
Página: 41
Requisição: Sucesso
Página não tem leads
Página: 42
Requisição: Sucesso
Página não tem leads
Página: 43
Requisição: Sucesso
Página não tem leads
Página: 44
Requisição: 

Requisição: Sucesso
Página não tem leads
Página: 37
Requisição: Sucesso
Página não tem leads
Página: 38
Requisição: Sucesso
Página não tem leads
Página: 39
Requisição: Sucesso
Página não tem leads
Página: 40
Requisição: Sucesso
Página não tem leads
Página: 41
Requisição: Sucesso
Página não tem leads
Página: 42
Requisição: Sucesso
Página não tem leads
Página: 43
Requisição: Sucesso
Página não tem leads
Página: 44
Requisição: Sucesso
Página não tem leads
Página: 45
Requisição: Sucesso
Página não tem leads
Página: 46
Requisição: Sucesso
Página não tem leads
Página: 47
Requisição: Sucesso
Página não tem leads
Página: 48
Requisição: Sucesso
Página não tem leads
Página: 49
Requisição: Sucesso
Página não tem leads
ID: 36
Página: 1
Requisição: Sucesso
Página tem leads
Página: 2
Requisição: Sucesso
Página não tem leads
Página: 3
Requisição: Sucesso
Página não tem leads
Página: 4
Requisição: Sucesso
Página não tem leads
Página: 5
Requisição: Sucesso
Página não tem leads
Página: 6
Requisição: Suc

Requisição: Sucesso
Página não tem leads
Página: 48
Requisição: Sucesso
Página não tem leads
Página: 49
Requisição: Sucesso
Página não tem leads
ID: 39
Página: 1
Requisição: Sucesso
Página não tem leads
Página: 2
Requisição: Sucesso
Página não tem leads
Página: 3
Requisição: Sucesso
Página não tem leads
Página: 4
Requisição: Sucesso
Página não tem leads
Página: 5
Requisição: Sucesso
Página não tem leads
Página: 6
Requisição: Sucesso
Página não tem leads
Página: 7
Requisição: Sucesso
Página não tem leads
Página: 8
Requisição: Sucesso
Página não tem leads
Página: 9
Requisição: Sucesso
Página não tem leads
Página: 10
Requisição: Sucesso
Página não tem leads
Página: 11
Requisição: Sucesso
Página não tem leads
Página: 12
Requisição: Sucesso
Página não tem leads
Página: 13
Requisição: Sucesso
Página não tem leads
Página: 14
Requisição: Sucesso
Página não tem leads
Página: 15
Requisição: Sucesso
Página não tem leads
Página: 16
Requisição: Sucesso
Página não tem leads
Página: 17
Requisição: Su

## De/Para das Bases

> Aqui vamos fazer o de/para da base do RD Station para a base do Exact Sales

### As bases

- Temos as duas bases de dados prontas, que foram extraídas via requisição na API de cada base
    - A base do RD Station foi tratada para ter no campo *custom_field* ligado ao *CNPJ* somente números, já que na extração original os dados vinham com caracteres especiais (,./-)

### Procedimento
- Agora, vamos pesquisar para cada lead da base RD se ela existe na base Exact utilizando o método *put* na requisição
    - **Se existe**: atualizar os dados que não estão preenchidos
    - **Se não existe**: criar o lead

In [20]:
import requests
import json

# Procurando CNPJ na base Rd
listRdOpen = open("completeDatabase_RD1.json")
listRd = json.load(listRdOpen)
#print(listRd)

for i in range(0, len(listRd[0])):
        for j in range(0, len(listRd[0][i]["custom_fields"]) - 1):
            #print(listRd[0][i])
            if (listRd[0][i]["custom_fields"][j]["custom_field"]["label"] == "CNPJ"):
                res = {k: v for k, v in listRd[0][i]["custom_fields"][j].items() if v == '15146185000114'}
                if (res):
                    result = res

print(result)
print("Finalizado")

{'value': '15146185000114'}
Finalizado
